# Manual implementation of MLP in TF
The implementation is on MNIST.

In [88]:
import tensorflow as tf
import numpy as np

n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

# Important!
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

### Create a function which generates a neural layer sub-graph when called

In [89]:
def neuron_layer(X, n_neurons, name, activation=None):
    """
    Create a sub-graph representing a neuron layer, 
    along with initialising the weights.
    
    Args:
        X (tf node): either the placeholder input or the output from another layer
        n_neurons (int): number of neurons
        name (strg): name of resulting node
        activation (tf function name): if None, just do a linear combination;
                          else apply the function name to the linear combination
    
    Returns:
        The activated output.
    
    """
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2/np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="kernal")
        
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        
        Z = tf.matmul(X, W) + b
        
        if activation is not None:
            return activation(Z)
        else:
            return Z


### Link multiple neuron layer sub-graphs together

In [90]:
with tf.name_scope("dnn"):
    hidden_1 = neuron_layer(X, 
                            n_hidden1, 
                            name="hidden1", 
                            activation=tf.nn.relu)
    hidden_2 = neuron_layer(hidden_1, 
                            n_hidden2, 
                            name="hidden2", 
                            activation=tf.nn.relu)
    logits = neuron_layer(hidden_2, 
                            n_outputs, 
                            name="outputs")


### Objective function

In [91]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")


### Optimisation

In [92]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)


### Evaluation

In [93]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

### Some operational steps

In [94]:
# Important!
init = tf.global_variables_initializer()
saver = tf.train.Saver()


### Now we're quite ready!

### Prepare data

In [95]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [96]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

### Run session

In [97]:
n_epochs = 4
batch_size = 50

with tf.Session() as sess:
    
    init.run()
    
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X:X_batch, y: y_batch})
        
        acc_batch = accuracy.eval(feed_dict={X:X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X:X_valid, y: y_valid})
        
        print(epoch, "Batch accuracy: ", acc_batch, "ValAccuracy: ", acc_val)
        
    save_path = saver.save(sess, "./my_model_final.ckpt")

            

0 Batch accuracy:  0.9 ValAccuracy:  0.9146
1 Batch accuracy:  0.92 ValAccuracy:  0.936


KeyboardInterrupt: 

# Separate test

In [ ]:
import tensorflow as tf

p = tf.placeholder(tf.float32, shape=(3))
a = tf.Variable(0.0)
b = tf.add(a, p)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    result = sess.run(b, feed_dict={p:[1, 1, 1]})
result